In [1]:
from init import *

### idx64_store

```python
def idx_store(idx, scope, table, payer, id, secondary) -> int
```


In [15]:
code = '''
import db
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    primary = 'hello'
    secondary = chain.s2n('world')
    itr, primary = chain.db_idx_find_primary(db.idx64, code, scope, table, primary)
    print(itr, primary)
    if itr >= 0:
        chain.db_idx_update(db.idx64, itr, payer, secondary)
    else:
        itr = chain.db_idx_store(db.idx64, scope, table, payer, primary, secondary)
'''

code = uuosapi.mp_compile(python_contract, code)

uuosapi.deploy_python_code(python_contract, code, deploy_type=1)
r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


0 16514440898579267584



### idx_find_primary

```python
def idx_find_primary(idx, code, scope, table, primary) -> [int, Union[int, float, float128]]
```

return [iter, secondary]

```
if idx == db.idx64 or idx == db.idx128 or idx == db.idx256:
    secondary is an int type
elif idx == db.idx_double
    secondary is an float type
elif idx == db.idx_long_double:
    secondary is an float128 type
```

In [17]:
code = '''
import db
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    primary = 'hello'
    itr, secondary = chain.db_idx_find_primary(db.idx64, code, scope, table, primary)
    print(itr, secondary)
'''

code = uuosapi.mp_compile(python_contract, code)

uuosapi.deploy_python_code(python_contract, code, deploy_type=1)
r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


0 16514440898579267584



### idx_find_secondary

```python
def idx_find_secondary(idx, code, scope, table, secondary) -> [int, int]
```

return [iter, primary]

In [18]:
code = '''
import db
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    secondary = chain.s2n('world')
    itr, primary = chain.db_idx_find_secondary(db.idx64, code, scope, table, secondary)
    print(itr, chain.n2s(primary))
'''

code = uuosapi.mp_compile(python_contract, code)

uuosapi.deploy_python_code(python_contract, code, deploy_type=1)
r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


0 hello



### idx_update

```python
def idx_update(idx, iterator, payer, secondary)
```


In [21]:
code = '''
import db
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    primary = 'hello'
    
    itr, secondary = chain.db_idx_find_primary(db.idx64, code, scope, table, primary)
    print(itr)
    if itr < 0:
        chain.db_idx_store(db.idx64, scope, table, payer, primary, b'world')
    else:
        data = chain.read_action_data()
        secondary = int.from_bytes(data, 'little')
        chain.db_idx_update(db.idx64, itr, payer, secondary)
        print(secondary)
'''

code = uuosapi.mp_compile(python_contract, code)

uuosapi.deploy_python_code(python_contract, code, deploy_type=1)
args = int.to_bytes(123, 8, 'little')
r = uuosapi.push_action(python_contract, 'sayhello', args, {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])



0
123



### idx_lowerbound

```python
def idx_lowerbound(idx, code, scope, table) -> [int, int, Union[int, float, float128]]
```

return [iter, primary, secondary]

### idx_upperbound

```python
def idx_upperbound(idx, code, scope, table) -> [int, int, Union[int, float, float128]]
```

return [iter, primary, secondary]


In [22]:
abi = ''
code = '''
import db
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    data = chain.read_action_data()
    if action == name('destroy'):
        while True:
            itr = chain.db_idx_end(db.idx64, code, scope, table)
            if itr == -1:
                break
            itr, primary = chain.db_idx_previous(db.idx64, itr)
            if itr < 0:
                break
            chain.db_idx_remove(db.idx64, itr)
        print('destroy done!')
    elif action == name('initdb'):
        chain.db_idx_store(db.idx64, scope, table, payer, 11, 1)
        chain.db_idx_store(db.idx64, scope, table, payer, 13, 3)
        chain.db_idx_store(db.idx64, scope, table, payer, 15, 5)
        chain.db_idx_store(db.idx64, scope, table, payer, 17, 7)
        print('init done!')
    elif action == name('test'):
        itr, primary, secondary = chain.db_idx_upperbound(db.idx64, code, scope, table, 3)
        print('upperbound of 3:', primary, secondary)

        itr, primary, secondary = chain.db_idx_upperbound(db.idx64, code, scope, table, 2)
        print('upperbound of 2:', primary, secondary)

        itr, primary, secondary = chain.db_idx_lowerbound(db.idx64, code, scope, table, 3)
        print('lowerbound of 3:', primary, secondary)

        itr, primary, secondary = chain.db_idx_lowerbound(db.idx64, code, scope, table, 2)
        print('lowerbound of 2:', primary, secondary)
'''

code = uuosapi.mp_compile(python_contract, code)

uuosapi.deploy_python_code(python_contract, code, deploy_type=1)

r = uuosapi.push_action(python_contract, 'destroy', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])

r = uuosapi.push_action(python_contract, 'initdb', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


destroy done!

init done!



In [25]:
r = uuosapi.push_action(python_contract, 'test', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


upperbound of 3: 15 5
upperbound of 2: 13 3
lowerbound of 3: 13 3
lowerbound of 2: 13 3




### idx_remove

```python
void idx_remove(iterator: int) -> int
```

### idx_next

```python
def idx_next(iterator: int) -> [int, int]
```

return [iter, primary]

### idx_previous

```python
def idx_previous(iterator: int, primary: int) -> [int, int]
```

return [iterator, primary]


### idx_end

```python
def idx_end(code, scope, table) -> int
```

In [26]:
abi = ''
code = '''
import db
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = chain.read_action_data()
    if action == name('destroy'):
        while True:
            itr = chain.db_end_i64(code, scope, table)
            if itr == -1:
                break
            itr, primary = chain.db_previous_i64(itr)
            if itr < 0:
                break
            chain.db_remove_i64(itr)

        while True:
            itr = chain.db_idx_end(db.idx64, code, scope, table)
            if itr == -1:
                break
            itr, primary = chain.db_idx_previous(db.idx64, itr)
            if itr < 0:
                break
            chain.db_idx_remove(db.idx64, itr)
        print('destroy done!')
    elif action == name('initdb'):
        chain.db_idx_store(db.idx64, scope, table, payer, 11, 1)
        chain.db_idx_store(db.idx64, scope, table, payer, 13, 3)
        chain.db_idx_store(db.idx64, scope, table, payer, 15, 5)
        chain.db_idx_store(db.idx64, scope, table, payer, 17, 7)
        print('init done!')
    elif action == name('test'):
        itr, primary, secondary = chain.db_idx_upperbound(db.idx64, code, scope, table, 3)
        print('upperbound of 3:', primary, secondary)

        itr, primary, secondary = chain.db_idx_upperbound(db.idx64, code, scope, table, 2)
        print('upperbound of 2:', primary, secondary)

        itr, primary, secondary = chain.db_idx_lowerbound(db.idx64, code, scope, table, 3)
        print('lowerbound of 3:', primary, secondary)

        itr, primary, secondary = chain.db_idx_lowerbound(db.idx64, code, scope, table, 2)
        print('lowerbound of 2:', primary, secondary)
'''

code = uuosapi.mp_compile(python_contract, code)

uuosapi.deploy_python_code(python_contract, code, deploy_type=1)

r = uuosapi.push_action(python_contract, 'destroy', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])

r = uuosapi.push_action(python_contract, 'initdb', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


destroy done!

init done!



In [27]:
r = uuosapi.push_action(python_contract, 'test', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


upperbound of 3: 15 5
upperbound of 2: 13 3
lowerbound of 3: 13 3
lowerbound of 2: 13 3

